In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import re
import pandas as pd

In [2]:
from multiprocessing import Pool

In [3]:
# https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/

In [4]:
empresas_listadas = pd.read_csv('InstrumentsConsolidatedFile_20220414_1.csv', sep=';')

/home/lucas/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (8,12,14,17,18,20,28,30,31,32,33,34,37,38,39,44,45,46,47,48,49,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def get_indicators(browser, path_indicators: str, path_values:str):
    df = pd.DataFrame(columns=['Ticker', 'Indicador'])

    indicadores = browser.find_element(by=By.XPATH, value=path_indicators)
    html_indicadores = indicadores.get_attribute('outerHTML')
    splited_indicadores = html_indicadores.split('data-name')
    for i in range(1, len(splited_indicadores)):
    #     print(splited_indicadores[i].split('=')[1].split('data-prefix')[0])
        df2 = {'Indicador': splited_indicadores[i].split('=')[1].split('data-prefix')[0].replace('"', '')}

        df = df.append(df2, ignore_index = True)


    valores = browser.find_element(by=By.XPATH, value=path_values)
    html_valores  = valores.get_attribute('outerHTML')
    splited_divs = html_valores.split('<div')
    # print(splited_divs)

    df_index = -1
    for i in range(1, len(splited_divs)):
    #     print(splited_divs[i])
        if ('tr w-100 d-flex justify-start asset-last' in splited_divs[i]):
    #         print('\n')
            is_header = False
            df_index +=1
            df_col = 2

        p = re.compile('>(.*)</')
        val = p.findall(splited_divs[i])

        if len(val) > 0:
            val = val[0].replace('</div>', '')
            if 'ATUAL' in val:
                is_header = True

            if is_header is True:
                df[val] = None

            else:
                df.iloc[df_index, df_col] = val
    #             print(val)
                df_col+=1
        
    return df

In [20]:
def get_company_indicators(ticker:str):
    try:
        driver = webdriver.Firefox("/usr/local/bin/")
        driver.get(f'https://statusinvest.com.br/acoes/{ticker}')

        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,"btn-close"))).click() # fecha o popup

        button = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]') # botão de histórico

        time.sleep(0.5)
        button.click()
        time.sleep(0.5)

        df_valuation = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[2]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[2]/div/div[2]/div')
        df_endividamento = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[3]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[3]/div/div[2]/div')
        df_eficiencia = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[4]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[4]/div/div[2]/div')
        df_rentabilidade = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[5]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[5]/div/div[2]/div')
        df_crescimento = get_indicators(driver, '/html/body/main/div[2]/div/div[7]/div[3]/div[6]/div/div[1]', '/html/body/main/div[2]/div/div[7]/div[3]/div[6]/div/div[2]/div')

        df = pd.concat([df_valuation, df_endividamento, df_eficiencia, df_rentabilidade, df_crescimento]).reset_index(drop=True)
        df['Ticker'] = ticker
        
        setor = driver.find_element(by=By.XPATH, value='/html/body/main/div[5]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong')
        setor = setor.get_attribute('outerHTML')
        df['Setor'] = setor.split('>')[1].split('<')[0]
        
        driver.close()
        
    except Exception as ex:
        print(ex)
        try:
            driver.close()
        except Exception:
            pass
        return None
    
    return df

In [36]:
tickers = empresas_listadas.loc[(empresas_listadas['TckrSymb'].str.len()>= 5) \
                                & (empresas_listadas['TckrSymb'].str.len()<= 6) \
                                & (~empresas_listadas['TckrSymb'].str.contains('^[0-9]')) \
                                & (~empresas_listadas['TckrSymb'].str.endswith('34'))\
                                & (~empresas_listadas['TckrSymb'].str.endswith('39'))\
                                & ((empresas_listadas['TckrSymb'].str.endswith('3')) | (empresas_listadas['TckrSymb'].str.endswith('3')) | (empresas_listadas['TckrSymb'].str.endswith('11')))
                                & (~empresas_listadas['TckrSymb'].str.endswith('T')), 'TckrSymb']
len(tickers)

1327

In [37]:
dataframes_list = []
with Pool(4) as p:
    dataframes_list = (p.map(get_company_indicators, tickers.values))

/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until


Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16

Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16



/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until


Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16



/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until


...

/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until


Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16



/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until
/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until


Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16

Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16

Message: Unable to locate element: /html/body/main/div[2]/div/div[7]/div[1]/div[2]/button[2]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:305:16



/home/lucas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
data = pd.concat(list(filter(lambda x: x is not None, dataframes_list)))

In [39]:
data.to_csv('data.csv')

In [40]:
# for ticker in tickers.values[:3]:
#     print(get_company_indicators(ticker))

In [41]:
data

,Ticker,Indicador,ATUAL,2021,2020,2019,2018,2017,2016,2015,2014,2013,Setor
0,AALR3,D.Y,-%,-%,"0,79%","0,47%",-%,-%,-%,-%,-%,-%,Serv.Méd.Hospit..Análises e Diagnósticos
1,AALR3,P/L,"-397,04","-314,82","-13,48","52,01","39,28","263,19",-,-,-,-,Serv.Méd.Hospit..Análises e Diagnósticos
2,AALR3,PEG Ratio,"4,21","3,34",-,-,"0,08",-,-,-,-,-,Serv.Méd.Hospit..Análises e Diagnósticos
3,AALR3,P/VP,"1,85","1,46","1,10","1,68","1,25","1,44",-,-,-,-,Serv.Méd.Hospit..Análises e Diagnósticos
4,AALR3,EV/EBITDA,"13,88","11,65","17,06","10,27","9,58","15,12",-,-,-,-,Serv.Méd.Hospit..Análises e Diagnósticos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,YDUQ3,ROA,"1,60%","1,60%","1,06%","11,72%","15,72%","10,56%","8,89%","10,10%","10,31%","11,44%",Serviços Educacionais
26,YDUQ3,ROIC,"5,70%","5,70%","3,27%","18,68%","22,40%","16,03%","13,14%","12,47%","11,88%","13,19%",Serviços Educacionais
27,YDUQ3,Giro ativos,"0,44","0,44","0,42","0,65","0,88","0,84","0,77","0,67","0,68","0,81",Serviços Educacionais
28,YDUQ3,CAGR Receitas 5 anos,"6,64%","6,64%","5,62%","8,38%","15,90%","19,56%","22,63%","23,60%","18,76%","12,05%",Serviços Educacionais


In [42]:
data.groupby('Ticker').size()

Ticker
AALR3    30
ABEV3    30
AERI3    30
AESB3    30
AFLT3    30
         ..
WEST3    30
WHRL3    30
WIZS3    30
WLMM3    30
YDUQ3    30
Length: 393, dtype: int64